In [ ]:
import pandas as pd
from fpdf import FPDF
from datetime import date
import os
import yfinance as yf
import matplotlib.pyplot as plt
import numpy as np
import cairosvg
import requests
import functions
from PIL import Image
from bs4 import BeautifulSoup
from bs4 import BeautifulSoup

In [ ]:
# Initialize the PDF object to create a new report
pdf = FPDF(orientation='P', unit='mm', format='A4')

In [ ]:
# Set up the font and add the first page to the PDF
pdf.set_font('Arial',)
pdf.add_page()

In [ ]:
# Add a visually appealing colored header background
pdf.set_fill_color(202, 164, 94)  # A gold-like color for the header
pdf.rect(0, 0, 210, 45, 'F')  # Draw a rectangle spanning the width of the page

In [ ]:
# Display the current date in white text at the top of the page
pdf.set_text_color(255, 255, 255)
pdf.set_font('Arial', 'I', 9)
pdf.cell(200, 10, date.today().strftime("%d %B %Y"), ln=True)

In [ ]:
# Add the main title of the report
pdf.set_font('Arial', 'B', 16)
pdf.ln(2)
pdf.cell(200, 10, 'HMC Capital Equity Research Report', ln=True)

In [ ]:
# Download and embed the HMC Capital logo into the PDF
svg_url = "https://www.hmccapital.com.au/themes/bs5/images/HMC-Capital-logo.svg"
response = requests.get(svg_url)

In [ ]:
if response.status_code == 200:
    with open("HMC_Capital_logo.svg", "wb") as f:
        f.write(response.content)
    cairosvg.svg2png(url="HMC_Capital_logo.svg", write_to="HMC_Capital_logo.png")
    pdf.image("HMC_Capital_logo.png", x=125, y=10, w=75)  # Position the logo on the top-right
    print("PDF created with converted SVG logo!")
else:
    print("Failed to download SVG logo.")

In [ ]:
# Add the authors' names to give credit to the contributors
pdf.set_font('Arial', '', 9)
pdf.ln(2)
pdf.cell(200, 10, 'By Flynn Hubbard, Jay Gills, Kirrtanya Tandon and Joshua Gray', ln=True)

In [ ]:
# Reset the text color to black for the main content of the report
pdf.set_text_color(0, 0, 0)

In [ ]:
# Add a "Company Overview" section to introduce the company
pdf.ln(6)
pdf.set_font('Arial', 'B', 16)
pdf.cell(200, 10, 'Company Overview', ln=True)

In [ ]:
# Provide a brief description of the company
pdf.set_font('Arial', '', 9)
company_overview = """
HMC Capital Limited (ASX: HMC) is a leading Australian alternative asset manager, specializing in high-conviction real asset strategies. Since rebranding from Home Consortium in 2021, it has rapidly expanded its funds under management to over $18.5Bn, focusing on real estate, private equity, private credit, energy transition, and digital infrastructure.
"""
pdf.multi_cell(95, 5, company_overview)

In [ ]:
# Align the "Analyst Price Targets" section to the right for better layout
pdf.set_xy(110, pdf.get_y() - 30)
pdf.set_font('Arial', 'B', 16)
pdf.set_xy(105, pdf.get_y() - 10)
pdf.set_text_color(202, 164, 94)  # Use a gold-like color for emphasis
pdf.cell(95, 5, 'BUY', ln=True, align='R')
pdf.set_text_color(0, 0, 0)

In [ ]:
# Add details about the stock's price target and other key metrics
pdf.set_font('Arial', '', 9)
price_targets = """
Stock Code: ASX:HMC
Price (as of 10/4/25): A$4.68
Price Target: A$9.00
Upside: 92.31%
Dividend yield: 2.56%
GICS Sector: Financials
"""
pdf.set_x(105)
pdf.multi_cell(95, 5, price_targets, align='R')

In [ ]:
# Add an "Industry Overview" section to provide context about the sector
pdf.ln(8)  # Adjusted spacing to move the section slightly higher
pdf.set_font('Arial', 'B', 16)
pdf.cell(200, 10, 'Industry Overview', ln=True)

In [ ]:
# Describe the alternative asset management industry and its trends
pdf.set_font('Arial', '', 9)
industry_overview = """
The alternative asset management industry has grown rapidly in recent years, driven by rising demand for diversification, higher returns, and protection against market swings. This sector, which includes real estate, private equity, infrastructure, and energy transition, is popular for its stable returns and long-term growth. Institutional investors, like pension funds and sovereign wealth funds, are increasingly shifting capital towards alternatives as traditional assets offer lower yields. ESG investing is also reshaping the industry, with firms incorporating sustainability into their strategies. Despite challenges like market volatility and competition, the industry offers strong growth opportunities, especially for firms focusing on emerging areas like digital infrastructure and renewable energy.
"""
pdf.multi_cell(0, 5, industry_overview)

In [ ]:
# Add a "DCF Output" section to present financial projections
pdf.ln(4)  # Adjusted spacing to move the section slightly higher
pdf.set_font('Arial', 'B', 16)
pdf.cell(200, 10, 'DCF Output', ln=True)

In [ ]:
# Read the DCF data from a CSV file and format it for the PDF
csv_file_path = "DCF CSV.csv"
df = pd.read_csv(csv_file_path)
df.rename(columns={'Unnamed: 0': 'Item'}, inplace=True)

In [ ]:
# Format and add the DCF table to the PDF
pdf.set_font('Arial', '', 9)
headers = ['AUD$m'] + [f'{year}E' for year in range(2025, 2030)]
pdf.ln(2)

In [ ]:
# Calculate column widths for a clean table layout
page_width = 200
left_margin = 10
right_margin = 10
usable_width = page_width - left_margin - right_margin
item_col_width = 70
number_col_width = (usable_width - item_col_width) / (len(headers) - 1)
row_height = 4

In [ ]:
# Add table headers with a distinct style
pdf.set_font('Arial', 'I', 9)
pdf.cell(item_col_width, row_height, headers[0], align='L')
pdf.set_font('Arial', 'B', 9)
pdf.set_text_color(202, 164, 94)  # Highlight headers with a gold-like color
for header in headers[1:]:
    pdf.cell(number_col_width, row_height, header, align='R')
pdf.set_text_color(0, 0, 0)
pdf.ln()
pdf.ln()

In [ ]:
# Populate the table with data from the CSV file
pdf.set_font('Arial', '', 9)
for i, (index, row) in enumerate(df.iterrows()):
    item_text = str(row['Item'])
    if item_text.startswith("Less:") or item_text.startswith("Add:"):
        item_text = "     " + item_text  # Indent subtotals for better readability
    pdf.cell(item_col_width, row_height, item_text, align='L')
    for col_index, value in enumerate(row[1:]):
        if pd.isna(value):
            cell_value = ""  # Leave empty cells for missing data
        else:
            if item_text.lower() == "shares outstanding":
                cell_value = f"{int(value):,}m"  # Format as millions
            elif item_text.lower() == "share price":
                cell_value = f"${float(value):.2f}"  # Format as currency
            else:
                cell_value = f"{int(value):,}"  # Format as integers with commas
        pdf.cell(number_col_width, row_height + 0.5, cell_value, align='R')
    pdf.ln()

In [ ]:
# Add a new page to the PDF
pdf.add_page()

In [ ]:
# Add a heading for the "Stock Price Performance" section
pdf.set_font('Arial', 'B', 16)
pdf.set_xy(10, 20)
pdf.cell(200, 10, 'Stock Price Performance', ln=True)

In [ ]:
# Fetch historical stock data for HMC and ASX200
ticker_hmc = 'HMC.AX'
ticker_asx200 = '^AXJO'
hmc = yf.download(ticker_hmc, start='2021-12-30', end='2025-04-27')
asx200 = yf.download(ticker_asx200, start='2021-12-30', end='2025-04-27')

In [ ]:
# Read analyst price targets from a CSV file
price_targets = pd.read_csv('price_targets.csv')
price_targets['Date'] = pd.to_datetime(price_targets['Date'])

In [ ]:
# Generate the first chart: HMC Share Price and Analyst Price Targets
fig1, ax1 = plt.subplots(figsize=(8, 6))
ax1.plot(hmc.index, hmc['Close'], label='HMC Share Price', color='blue')  # Plot HMC share price
ax1.plot(price_targets['Date'], price_targets['Target Price'], linestyle='--', marker='o', color='black', label='Analyst Price Targets')  # Overlay analyst price targets
ax1.set_xlabel('Date')  # Set x-axis label
ax1.set_ylabel('HMC Price (AUD)', color='blue')  # Set y-axis label with matching color
ax1.tick_params(axis='y', labelcolor='blue')  # Match y-axis tick color to the line
ax1.set_title('HMC Share Price and Analyst Targets', pad=20)  # Add a title with padding
ax1.legend(loc='upper left')  # Place the legend in the upper left corner
ax1.grid(True)  # Enable grid for better readability
ax1.xaxis.set_major_locator(plt.MaxNLocator(6))  # Limit the number of x-axis ticks for clarity
graph1_path = 'hmc_price_targets_chart.png'  # Save the chart as a PNG file
plt.savefig(graph1_path)
plt.close()  # Close the figure to free up memory

In [ ]:
# Generate the second chart: HMC Share Price vs ASX200 Index
fig2, ax2 = plt.subplots(figsize=(8, 6))
ax2.plot(hmc.index, hmc['Close'], label='HMC Share Price', color='blue')  # Plot HMC share price
ax2.set_xlabel('Date')  # Set x-axis label
ax2.set_ylabel('HMC Price (AUD)', color='blue')  # Set left y-axis label
ax2.tick_params(axis='y', labelcolor='blue')  # Match left y-axis tick color to the line

In [ ]:
# Add a secondary y-axis for the ASX200 Index
ax3 = ax2.twinx()
ax3.plot(asx200.index, asx200['Close'], label='ASX200 Index', color='grey', linestyle='--')  # Plot ASX200 index
ax3.set_ylabel('ASX200 Index Level', color='grey')  # Set right y-axis label
ax3.tick_params(axis='y', labelcolor='grey')  # Match right y-axis tick color to the line

In [ ]:
# Add a combined title and legend for both axes
fig2.suptitle('HMC Share Price and ASX200 Index', y=0.95)  # Set the title with adjusted vertical position
lines1, labels1 = ax2.get_legend_handles_labels()  # Get legend details for the first axis
lines2, labels2 = ax3.get_legend_handles_labels()  # Get legend details for the second axis
ax2.legend(lines1 + lines2, labels1 + labels2, loc='upper left')  # Combine legends and place in the upper left corner
ax2.grid(True)  # Enable grid for better readability
ax2.xaxis.set_major_locator(plt.MaxNLocator(6))  # Limit the number of x-axis ticks for clarity
graph2_path = 'hmc_asx200_chart.png'  # Save the chart as a PNG file
plt.savefig(graph2_path)
plt.close()  # Close the figure to free up memory

In [ ]:
# Embed the generated charts into the PDF
pdf.image(graph1_path, x=10, y=30, w=95)  # Add the first chart on the left side of the page
pdf.image(graph2_path, x=110, y=30, w=95)  # Add the second chart on the right side of the page

In [ ]:
# Add small headers for branding and context
pdf.set_font('Arial', '', 9)
pdf.set_xy(10, 10)  # Position for the left header
pdf.cell(0, 5, 'FINM3422 Group 90', ln=False)  # Add group name
pdf.set_xy(200 - pdf.get_string_width('HMC Capital Equity Research Report') - 10, 10)  # Position for the right header
pdf.cell(0, 5, 'HMC Capital Equity Research Report', ln=False)  # Add report title

In [ ]:
# Add a heading for the "Recent Headlines" section
pdf.set_xy(10, 120)  # Position below the two charts
pdf.set_font('Arial', 'B', 16)  # Set font size to 16 for the heading
pdf.cell(200, 10, 'Recent Headlines', ln=True)

In [ ]:
# Add a small gap between the heading and the body text
pdf.ln(2)

In [ ]:
# Fetch recent headlines from the AFR website
AFR = 'https://www.afr.com/company/asx/hmc'

In [ ]:
try:
    RawHTML = requests.get(AFR, timeout=10)  # Add a timeout to avoid hanging
    RawHTML.raise_for_status()  # Raise an HTTPError for bad responses (4xx and 5xx)
    # Parse the raw HTML using BeautifulSoup
    parse = BeautifulSoup(RawHTML.content, 'html.parser')

    # Extract recent headlines specific to the AFR format
    headlines = parse.find_all('h3', class_='-_73414cb90844eda9-headline')

    # Get the top 7 headlines and clean the text
    recent_headlines = [headline.get_text(strip=True) for headline in headlines[:10]]
    
    # Remove or replace non-ASCII characters
    recent_headlines = [headline.encode('ascii', 'ignore').decode('ascii') for headline in recent_headlines]
    
    if not recent_headlines:
        recent_headlines = ["No headlines found."] * 7  # Fallback if no headlines are found
except (requests.RequestException, Exception) as e:
    print(f"Error fetching or parsing headlines: {e}")
    recent_headlines = ["Error fetching headlines."] * 7  # Fallback in case of an error

In [ ]:
# Add the headlines to the PDF as a numbered list
pdf.set_font('Arial', '', 9)
for i, headline in enumerate(recent_headlines, start=1):
    pdf.cell(190, 5, f"{i}. {headline}", ln=True)  # Adjust width for smaller right margin

In [ ]:
# Add a "Financial Ratios" table
pdf.set_xy(10, 190)  # Adjust position to fit the table on the page

In [ ]:
# Add a title for the table
pdf.set_font('Arial', 'B', 16)
pdf.cell(100, 10, 'Financial Ratios', ln=True)

In [ ]:
# Add table headers
pdf.set_font('Arial', 'B', 9)
pdf.set_fill_color(202, 164, 94)
pdf.set_text_color(255, 255, 255)
pdf.cell(45, 6, 'Metric', border=1, fill=True)  # Column for metric names
pdf.cell(45, 6, 'Value', border=1, fill=True)  # Column for metric values
pdf.ln()

In [ ]:
# Add table rows with financial ratios
pdf.set_font('Arial', '', 9)
pdf.set_text_color(0, 0, 0)
for index, row in functions.df_ratios.iterrows():
    metric = str(row['Metric'])
    try:
        value = f"{float(row['Value']):.2f}"  # Format values to two decimal places
    except ValueError:
        value = str(row['Value'])  # Fallback for non-numeric values
    pdf.cell(45, 6, metric, border=1)  # Add metric name
    pdf.cell(45, 6, value, border=1)  # Add metric value
    pdf.ln()

In [ ]:
# Add a "Recommendation" section aligned with the "Recent Headlines" heading
pdf.set_xy(110, 120)  # Position to the right of the page
pdf.set_font('Arial', 'B', 16)
pdf.cell(90, 10, 'Recomendations', ln=True)

In [ ]:
# Add the investment overview text
pdf.set_font('Arial', '', 9)
pdf.set_xy(110, 130)  # Adjust position for the body text
investment_overview = """
HMC Capital Ltd presents a compelling investment opportunity within the Australian alternative asset management sector, thanks to its strong fundamentals and strategic initiatives that position it for impressive long-term growth. Despite the recent market volatility impacting its share price, the company continues to demonstrate resilience and solid performance. In the first half of FY25, HMC reported a record pre-tax operating earnings of $202.2 million, a staggering 240% increase from the previous year. Assets under management surged to $18.5 billion, reflecting a remarkable 45% rise since June 2024. This signifies the success of HMC's expansion strategies and growing investor confidence.

However, the business does face notable risks. These include the inherent volatility of the market, increased competition within the asset management space, and execution risks as it expands its fund offerings. Additionally, regulatory changes, particularly those related to ESG investing, could affect its strategies and future growth potential. Another risk is the possibility of failing to meet return expectations for new funds, which could harm the company's reputation and investor trust. Despite this, Goldman Sachs initiated coverage with a 'Buy' rating and a price target of $8.94, noting that HMC's scalable real asset strategies and strong structural growth drivers make it an attractive investment. Overall, despite the risks, this stock has strong growth prospects, making it a buy recommendation for long-term investors.
"""
pdf.multi_cell(90, 5, investment_overview, align='L')

In [ ]:
# Save the final PDF report
pdf.output("HMC Capital Equity Research Report.pdf")